In [ ]:
from crewai import Agent, Task, Crew, Process
from crewai.llm import LLM
import os
from dotenv import load_dotenv
from IPython.display import Markdown

load_dotenv()

# Initialize Groq LLM with DeepSeek-R1-Distill-Llama-70B
llm = LLM(
    model="groq/deepseek-r1-distill-llama-70b",
    api_key=os.getenv("GROQ_API_KEY")
)

planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate LinkedIn posts on technical topics that are easy to digest.",
    backstory=(
        "You specialize in creating engaging, bite-sized LinkedIn posts for technical topics. "
        "Your mission is to make complex concepts accessible and interesting, especially for beginners. "
        "You focus on quick, actionable insights that spark curiosity and encourage learning, "
        "ensuring each post is both informative and easy to understand. "
        "Your content planning is tailored to help readers gain valuable knowledge without feeling overwhelmed."
        "Your work is the basis for the Content Writer to write an article on this topic.",
    ),
    allow_delegation=False, 
    verbose=True,
    llm=llm
)

writer = Agent(
    role="Content Writer",
    goal="Write a LinkedIn post on {topic} based on the plan provided by the Content Planner, possibly including character-based diagrams and simple analogies.",
    backstory=(
        "You are a skilled technical writer with expertise in crafting clear, engaging LinkedIn posts. You have a special talent for simplifying complex topics using character-based diagrams (like flowcharts) and relatable analogies and concise writing. "
        "Your mission is to take the plan provided by the Content Planner and transform it into a well-structured, "
        "informative, and interesting short-form LinkedIn post. "
        "You focus on delivering valuable insights in a punchy and concise format, ideal for grabbing attention quickly, ensuring each post is both informative and easy to understand."
    ),
    allow_delegation=False,
    verbose=True,
    llm=llm
)

reviewer = Agent(
    role="Content Reviewer",
    goal="Review the LinkedIn post for quality, accuracy, and engagement, providing feedback for improvement.",
    backstory=(
        "You are an experienced content reviewer with a keen eye for detail and a deep understanding of what makes a LinkedIn post successful. "
        "Your mission is to ensure that every post is polished, professional, and aligns with the intended goals. "
        "You check for clarity, tone, factual accuracy, and overall impact, providing constructive feedback to elevate the content to its highest potential."
    ),
    allow_delegation=False,
    verbose=True,
    llm=llm
)

# Create tasks for each agent
plan_task = Task(
    description=(
        "Create a detailed content plan for a LinkedIn post on the topic: {topic}. "
        "The plan should be engaging, factually accurate, and easy for beginners to digest. Include key points, a suggested structure, and any relevant hashtags. "
        "Also, suggest if a simple character-based diagram or a short analogy would be beneficial for explaining the topic."
    ),
    expected_output="A comprehensive content plan in markdown format, including an outline, key talking points, relevant hashtags, and a suggestion on whether to use a character-based diagram or analogy.",
    agent=planner
)

write_task = Task(
    description=(
        "Using the content plan from the planner, write a compelling LinkedIn post on {topic}. The post should be clear, engaging, and well-structured. "
        "If the topic is complex, consider using a simple character-based diagram or a short analogy to make it easier to understand. "
        "Make sure to follow the provided plan and maintain a professional yet accessible tone."
    ),
    expected_output="A short, punchy LinkedIn post as a markdown text, ready to be published. The post should be concise, ideally 1-2 paragraphs long, and may include a simple character-based diagram or analogy if it helps with clarity.",
    agent=writer,
    context=[plan_task],
)

review_task = Task(
    description=(
        "Review the LinkedIn post written by the Content Writer. "
        "Check for quality, accuracy, clarity, and engagement. "
        "Provide constructive feedback and make necessary edits to ensure the post is polished and ready for publication."
    ),
    expected_output="The final, approved version of the LinkedIn post as a markdown text, with all feedback incorporated.",
    agent=reviewer,
    context=[write_task],
)


crew = Crew(
    agents=[planner, writer, reviewer],
    tasks=[plan_task, write_task, review_task], 
    verbose=2,
    process=Process.sequential,
)

if __name__ == "__main__":
    result = crew.kickoff(inputs={"topic": "System design interview questions"})
    print(Markdown(result))